In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import pandas
import seaborn as sns

import scipy
import scipy.cluster.hierarchy as hac

import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

from ipywidgets import interact, widgets

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Fonction de chargement brutes des données issus du fichier '1335.vdb.tab'
def get_raw_data():
    data = pandas.read_table('1335.vdb.tab', sep = '\t', header = 0)
    data = data.sort_values(by = ['OTUConTaxonomy'])
    data = data.reset_index(drop=True)

    list_taxon=['Domaine','Embranchement','Classe','Ordre',"Famille","Genre"]
    
    for i in range(6):
        colname = list_taxon[i]
        data[colname]=data.apply(lambda row:row.OTUConTaxonomy.split(";")[i].split("(")[0],axis=1)

    data['OTU'] = data.apply(lambda row:row.OTUNumber.replace('Otu',''), axis=1)
    data = data.drop(columns=['OTUConTaxonomy','OTUNumber'])
    data.columns=['Door1','Door2','FaucetHandle1','FaucetHandle2','SinkFloor1','SinkFloor2',
                  'Soap Dispenser','Stall','ToiletFloor1','ToiletFloor2','ToiletFlushHandle1',
                  'ToiletFlushHandle2','ToiletSeat1','ToiletSeat2','Domaine','Embranchement',
                  'Classe','Ordre','Famille','Genre','OTU']
    
    return data

row_data = get_raw_data() # Obtention des data brutes refactorisées

In [ ]:
# Figure 1 : Clustering hierarchique sur les données brutes
def get_hac(data, label):
    Z = hac.linkage(data, method='complete', metric='correlation', optimal_ordering=False)

    plt.figure(figsize=(18, 10))
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('Sample name')
    plt.ylabel('Distance')
    
    hac.dendrogram(Z, leaf_rotation=45., leaf_font_size=12., labels = label)
    
    plt.show()

data_full = pandas.DataFrame(data = row_data, columns = ['Door1','Door2','FaucetHandle1','FaucetHandle2',
                                                         'SinkFloor1','SinkFloor2','Soap Dispenser','Stall',
                                                         'ToiletFloor1','ToiletFloor2','ToiletFlushHandle1',
                                                         'ToiletFlushHandle2','ToiletSeat1','ToiletSeat2'])
get_hac(data_full.T,data_full.columns)

In [ ]:
# Fonction de construction d'un jeu de donnée plus élaboré : 
# refactoring des données brutes pour mieux visualiser des corrélations.
# On se base sur les embranchements au niveaux taxonomique avec un seuil d'expression de reads supérieur à 10
def get_dataset(data_set, section, threshold):
    data = pandas.pivot_table(data_set, columns = section, aggfunc='sum')
    vec = data.apply(lambda col:col.sum()>=threshold, axis=0)

    data = data.loc[:,vec]
    data = pandas.DataFrame.transpose(data)
    
    return data

Section = 'Embranchement'
data = get_dataset(row_data, Section, 10)

In [ ]:
# Figure 2 : PCA sur le jeu de données brutes et le jeu de données refactorisés
def get_PCA(data,Section):
    data_PCA = data.values
    data_PCA = StandardScaler().fit_transform(data_PCA)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(data_PCA)

    #print("Pourcentage de variance expliquée : ")
    #print(pca.explained_variance_ratio_)

    principalDf = pandas.DataFrame(data = principalComponents, columns = ['principal component 1', 
                                                                          'principal component 2'])
    tagretDf = pandas.DataFrame(data = data.columns, columns = ['target'])
    finalDf = pandas.concat([principalDf, tagretDf[['target']]], axis = 1)

    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    
    ax.set_xlabel('Principal Component 1 : {} %'.format(round(pca.explained_variance_ratio_[0]*100,2)), 
                  fontsize = 15, labelpad = 15)
    ax.set_ylabel('Principal Component 2 : {} %'.format(round(pca.explained_variance_ratio_[1]*100,2)), 
                  fontsize = 15, labelpad = 15)
    ax.set_title('2 Component PCA : {}'.format(Section), fontsize = 20, pad = 20)
    
    for target in data.columns:
        indicesToKeep = finalDf['target'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1'], 
                   finalDf.loc[indicesToKeep, 'principal component 2'], 
                   c = 'black', s = 30 )

        plt.annotate(target,(finalDf[indicesToKeep]['principal component 1'],
                             finalDf[indicesToKeep]['principal component 2']),
                     textcoords="offset points", xytext=(0,10), ha='center', size=10)

# PCA sur le dataset entier
get_PCA(data_full,'All')

# PCA sur un dataset spécifique
get_PCA(data, Section)

In [ ]:
# Figure 3 : Heatmap de corrélation d'expression de reads entre les samples à partir du jeu de données refactorisées
def get_heatmap(data, section):
    fig, ax = plt.subplots(figsize=(12, 10))

    df_corr = data.corr()
    np.ones_like(df_corr, dtype=np.bool)
    mask = np.triu(np.ones_like(df_corr, dtype=np.bool))
    mask = mask[1:, :-1]
    
    corr = df_corr.iloc[1:,:-1].copy()

    sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap="Blues",
               linewidths=0.3, vmin=0, vmax=1, cbar=False,
               cbar_kws={"shrink": .8}, square=True)

    yticks = [i for i in corr.index]
    xticks = [i for i in corr.columns]

    plt.yticks(plt.yticks()[0], labels=yticks, rotation=0)
    plt.xticks(plt.xticks()[0], labels=xticks)

    plt.title('Expression level : Sample / Sample for the section "{}"'.format(section), loc='center')

get_heatmap(data, Section)

In [ ]:
# Figure 4 : Clustering hierarchique et heatmap de corrélation entre sample et embranchement prélevés
def get_clustering(data_set, section):
    sns.clustermap(data_set, cmap="Blues", vmin= 0, vmax=2800,metric='euclidean',
                   figsize=(10, 10),dendrogram_ratio=0.3,colors_ratio=0.003,cbar_pos=(0.02, 0.8, 0.05, 0.18),
                   linewidth=0.3, cbar_kws={"shrink": .8})

    plt.xlabel('Sample')
    plt.ylabel(section)

    plt.title('Expression level : {} / Sample'.format(section), loc='left')

get_clustering(data, Section)

In [ ]:
# Figure 2, 3 et 4 en interactif : Possibilité de faire varier le seuil minimal d'expression 
# et le niveau taxonomique observé
@widgets.interact_manual(Threshold=(0,3000,100), Section=['Embranchement','Classe','Ordre','Famille','Genre','OTU'])
def show_correlation(Threshold,Section):
    data_full = get_dataset(row_data, Section, Threshold)
    
    get_PCA(data_full, Section)
    get_heatmap(data_full, Section)
    get_clustering(data_full, Section)

In [ ]:
#Figure 5 : Sunburst pour modéliser la présence des espèces par leur expression dans chacune des conditions

data = get_raw_data()

sample_choice = ['Door1','Door2','FaucetHandle1','FaucetHandle2','SinkFloor1','SinkFloor2','Soap Dispenser',
                 'Stall','ToiletFloor1','ToiletFloor2','ToiletFlushHandle1','ToiletFlushHandle2','ToiletSeat1',
                 'ToiletSeat2']

sample = sample_choice[0]

fig = px.sunburst(row_data, path=['Domaine','Embranchement','Classe','Ordre','Famille','Genre'], 
                  values=round((data[sample]/sum(data[sample]))*100,2),
                  color='Embranchement', branchvalues='total',template='ggplot2',title='sunburst',
                  color_discrete_sequence=px.colors.qualitative.Pastel)

fig.update_traces(textinfo='label+percent entry')
fig.update_layout(margin=dict(t=50,l=0,r=0,b=50))
fig.show()

In [ ]:
#Figure 5.1 : Sunburst par paire en mode statique

sample='ToiletSeat'

def make_figure(id_sample): 
    sb1 = px.sunburst(data, path=['Domaine','Embranchement','Classe','Ordre','Famille','Genre'], 
                      values=round((data[str(id_sample)+"1"]/sum(data[str(id_sample)+"1"]))*100,2),
                      color='Embranchement', branchvalues='total',template='ggplot2',title='sunburst',
                      color_discrete_sequence=px.colors.qualitative.Pastel)._data

    sb2 = px.sunburst(data, path=['Domaine','Embranchement','Classe','Ordre','Famille','Genre'], 
                      values=round((data[str(id_sample)+"2"]/sum(data[str(id_sample)+"2"]))*100,2),
                      color='Embranchement', branchvalues='total',template='ggplot2',title='sunburst',
                      color_discrete_sequence=px.colors.qualitative.Pastel)._data

    trace1 = go.Sunburst(ids=sb1[0]['ids'],
                         labels=sb1[0]['labels'],
                         values=sb1[0]['values'],
                         parents=sb1[0]['parents'],
                         domain={'x': [0.0, 0.5], 'y': [0, 1]},
                         branchvalues ="total")

    trace2 = go.Sunburst(ids=sb2[0]['ids'],
                         labels=sb2[0]['labels'],
                         values=sb2[0]['values'],
                         parents=sb2[0]['parents'],
                         domain={'x': [0.5, 1.0], 'y': [0, 1]},
                         branchvalues ="total")

    layout = go.Layout(height = 500,
                       width = 950,
                       autosize = False,
                       title = 'Side by side Sunburst diagrams by kind of paired sample {} : Sample 1 vs Sample 2'.format(id_sample))

    fig = go.Figure(data = [trace1, trace2], layout = layout)
    fig.update_traces(textinfo='label+percent entry')

    return fig

make_figure(sample)

In [ ]:
# Figure 5.2 : Sunburst interactif pour tout les échantillons par paires via dash app
# S'il n'apparait pas, le rendu finale de cette figure consiste en l'ajout d'un dropdown à la fiure précédente
# afin de choisir la paire de sample à comparer
app = JupyterDash(__name__)

sample_choice = ['Door','FaucetHandle','SinkFloor','ToiletFloor','ToiletFlushHandle','ToiletSeat']

app.layout = html.Div([
    dcc.Graph(figure=make_figure(sample_choice[0]), id='graph'),
    html.Label([
        "Sample",
        dcc.Dropdown(
            id='sample-choice', clearable=False,
            value='Door', options=[
                {'label': c, 'value': c}
                for c in sample_choice
            ])
    ]),
])

@app.callback(
    Output('graph', 'figure'),
    [Input("sample-choice", "value")]
)
def update_figure(sample): 
    return make_figure(sample)

app.run_server(mode='inline', port=8000)

In [ ]:
#Shutdown l'app précédente
@classmethod
def _terminate_server_for_port(cls, host, port):
    shutdown_url = "http://{host}:{port}/_shutdown_{token}".format(
        host=host, port=port, token=JupyterDash._token
    )
    try:
        response = requests.get(shutdown_url)
    except Exception as e:
        pass

app._terminate_server_for_port("localhost", 8000)

In [ ]:
# Ajout d'un wiget interactif : Choisir les type(s) d'oragnisme(s) d'intérêt(s) qu'on souhaite extraire
# du dataset brute initital

# Les organismes sont extrait au niveaux des embranchements. Un moteur de recherche est mis à disposition
# de l'utilisateur pour simplifier sa recherche. Un ou plisieurs embranchements peuvent être extraits
# dans un fichier .csv  et téléchargés dans le dosser /sampling à la racine de ce projet.

def multi_checkbox_widget(descriptions):
    search_widget = widgets.Text()
    options_dict = {description: widgets.Checkbox(description=description, value=False) for description in descriptions}
    options = [options_dict[description] for description in descriptions]
    options_widget = widgets.VBox(options, layout={'overflow': 'scroll'})
    multi_select = widgets.VBox([search_widget, options_widget])
    print("Choix des organismes disponibles et moteur de recherche : ")
    
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            new_options = [options_dict[description] for description in descriptions]
        else:
            close_matches = difflib.get_close_matches(search_input, descriptions, cutoff=0.0)
            new_options = [options_dict[description] for description in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    
    return multi_select

print("Widget pour l'extraction d'un sous ensemble d'embranchement à partir du jeu de donnée brute.")
button = widgets.Button(description="Download CSV")
output = widgets.Output()
display(button, output)

widget = multi_checkbox_widget(row_data['Embranchement'].unique()) 
display(widget)

@output.capture()
def on_button_clicked(b):
    out=[w.description for w in widget.children[1].children if w.value]

    row_data = get_raw_data()
    final_data=pandas.DataFrame(columns=row_data.columns)
    final_name=[]
    
    for i in range(len(out)):
        organism = row_data['Embranchement'] == out[i]
        frames=[final_data,row_data[organism]]
        final_data = pandas.concat(frames)
        final_name.append(str(out[i]+"_"))
    
    file_name = "".join(final_name).strip("_")
    final_data.to_csv('sampling/{}.csv'.format(file_name), sep='\t')
    
    print("Le fichier {}.csv à bien été téléchagé dans le dossier /sampling.".format(file_name))

button.on_click(on_button_clicked)

In [ ]:
# Figure 6 : Barplot interactif utilisable pour visualiser un sous jeu de donnée d'intérêt
# extrait à partir du widget précédent
data = pandas.read_csv('sampling/Actinobacteria_Proteobacteria.csv', sep = '\t')
data = pandas.DataFrame(data, columns = ['Door1','Door2','FaucetHandle1','FaucetHandle2','SinkFloor1','SinkFloor2','Soap Dispenser','Stall','ToiletFloor1','ToiletFloor2','ToiletFlushHandle1','ToiletFlushHandle2','ToiletSeat1','ToiletSeat2','Domaine','Embranchement','Classe','Ordre','Famille','Genre'])

res_list={}
list_taxon=['Domaine','Embranchement','Classe','Ordre'] #"Famille" et "Genre" peuvent être ajoutés mais non pertinant

for taxon in list_taxon[1:]:
    res = pandas.pivot_table(data, columns = taxon, aggfunc='sum')
    mask = res.apply(lambda col:col.sum()>=10, axis=0)

    res = res.loc[:,mask]
    res_list[taxon] = res
    
w = widgets.Dropdown(
    options=list_taxon[1:],
    value='Embranchement',
    description='Taxon :'
)

@widgets.interact(tax=w)
def plot_func(tax):
    ax = res_list[tax].plot(kind="bar",stacked=True, legend = True)
    ax.legend(bbox_to_anchor=(1,1), loc="upper left")
    ax.set_title('Barplot représentant l\'expression des embranchements \n Actinobacteria et Proteobacteria dans les différents samples.')